In [1]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd
from sklearn.linear_model import LinearRegression
# fetch dataset 
air_quality = fetch_ucirepo(id=360) 
# data (as pandas dataframes) 
x = air_quality.data.features 
# none
y = air_quality.data.targets 

x["datetime"] = pd.to_datetime(x["Date"] + " " + x["Time"])

x = x.drop(columns=['Date', 'Time'])
x.where(x != -200, inplace=True)
x.dropna(inplace=True)
x.describe()

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,datetime
count,827.000000,827.000000,827.000000,827.000000,827.000000,827.000000,827.000000,827.000000,827.000000,827.000000,827.000000,827.000000,827.000000,827
mean,2.353567,1207.879081,231.025393,10.771100,966.116082,143.501814,963.297461,100.259976,1600.620314,1045.812576,15.601451,49.050181,0.831853,2004-04-04 14:58:02.466747264
min,0.300000,753.000000,7.000000,0.500000,448.000000,12.000000,461.000000,19.000000,955.000000,263.000000,6.300000,14.900000,0.402300,2004-03-10 18:00:00
25%,1.300000,1017.000000,77.000000,4.800000,754.000000,81.000000,769.000000,78.500000,1369.500000,760.000000,11.900000,36.700000,0.718950,2004-03-24 11:30:00
50%,2.000000,1172.000000,157.000000,9.100000,944.000000,128.000000,920.000000,99.000000,1556.000000,1009.000000,15.000000,49.600000,0.817700,2004-04-02 18:00:00
75%,3.100000,1380.000000,318.500000,14.800000,1142.500000,187.000000,1131.000000,122.000000,1783.500000,1320.000000,18.300000,60.550000,0.927500,2004-04-15 16:30:00
max,8.100000,2040.000000,1189.000000,39.200000,1754.000000,478.000000,1935.000000,196.000000,2679.000000,2359.000000,30.000000,83.200000,1.485200,2004-05-01 00:00:00
std,1.409496,241.816997,208.461912,7.418134,266.424557,81.829717,265.914168,31.493823,302.291793,400.134662,4.825304,15.266746,0.178506,NaN


Get useful features to predict NOx(GT)

In [2]:
useful_features = ['NOx(GT)', 'CO(GT)', 'PT08.S1(CO)', 'C6H6(GT)', 'PT08.S2(NMHC)', 'PT08.S4(NO2)']
cropped = x.filter(useful_features)
cropped.describe()


,NOx(GT),CO(GT),PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),PT08.S4(NO2)
count,827.000000,827.000000,827.000000,827.000000,827.000000,827.000000
mean,143.501814,2.353567,1207.879081,10.771100,966.116082,1600.620314
std,81.829717,1.409496,241.816997,7.418134,266.424557,302.291793
min,12.000000,0.300000,753.000000,0.500000,448.000000,955.000000
25%,81.000000,1.300000,1017.000000,4.800000,754.000000,1369.500000
50%,128.000000,2.000000,1172.000000,9.100000,944.000000,1556.000000
75%,187.000000,3.100000,1380.000000,14.800000,1142.500000,1783.500000
max,478.000000,8.100000,2040.000000,39.200000,1754.000000,2679.000000


Separate training and test data

In [3]:
training = cropped.sample(frac=0.8)
training

,NOx(GT),CO(GT),PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),PT08.S4(NO2)
512,50.0,1.1,1034.0,4.1,716.0,1405.0
97,261.0,4.6,1808.0,20.6,1312.0,1993.0
1095,48.0,0.8,875.0,2.3,603.0,1206.0
305,220.0,3.0,1308.0,15.5,1166.0,1794.0
175,92.0,1.4,1161.0,8.1,904.0,1502.0
...,...,...,...,...,...,...
521,130.0,1.6,1094.0,6.8,850.0,1489.0
300,33.0,0.5,909.0,1.8,567.0,1287.0
1044,72.0,0.8,923.0,4.9,757.0,1418.0
18,104.0,1.6,1236.0,5.2,774.0,1401.0


In [4]:
test = cropped.copy()
test = pd.merge(test, training, how='outer',indicator=True).query('_merge == "left_only"').drop(columns=['_merge'])
test

,NOx(GT),CO(GT),PT08.S1(CO),C6H6(GT),PT08.S2(NMHC),PT08.S4(NO2)
8,17.0,0.4,803.0,1.3,527.0,1152.0
10,18.0,0.5,858.0,0.9,490.0,1212.0
11,19.0,0.5,846.0,1.6,557.0,1212.0
13,21.0,0.4,830.0,1.4,538.0,1214.0
15,22.0,0.3,824.0,0.8,478.0,1200.0
...,...,...,...,...,...,...
818,377.0,6.6,1852.0,36.4,1696.0,2535.0
819,378.0,6.2,1722.0,31.9,1595.0,2439.0
820,383.0,6.9,1776.0,27.4,1488.0,2333.0
821,394.0,5.8,1771.0,26.6,1470.0,2262.0


In [5]:
predictor = LinearRegression().fit(training.iloc[:, 1:], training.iloc[:, 0])
# R^2
predictor.score(test.iloc[:, 1:], test.iloc[:, 0])

0.9192750947615995